## Learning objectives

* Imrpove a linear regression model with the addition of additional synthetic features (this is a continuation of the previous excercise)
* Use an input function to convert pandas `DataFrame` objects to `Tensors` and invoke the input function in `fit()` and `predict()` operations
* Use the FTRL optimization algorithm for model training
* Create new synthetic features through one-hot encoding, binning, and feature crosses


# Setup

Load data first


In [2]:
from __future__ import print_function

import math 

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

from tensorflow.keras import optimizers
from functools import partial


# Original Codelab code had this line, but it is not compatible with TensorFlow 2
# tf.logging.set_verbosity(tf.logging.ERROR)
# Replaced with below
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

# Line below was missing - i.e. we did not reshuffle the data.
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index)
)

In [3]:
def preprocess_features(california_housing_dataframe):
  """Prepares input features from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """
  selected_features = california_housing_dataframe[
    ["latitude",
     "longitude",
     "housing_median_age",
     "total_rooms",
     "total_bedrooms",
     "population",
     "households",
     "median_income"]]
  processed_features = selected_features.copy()
  # Create a synthetic feature.
  processed_features["rooms_per_person"] = (
    california_housing_dataframe["total_rooms"] /
    california_housing_dataframe["population"])
  return processed_features

def preprocess_targets(california_housing_dataframe):
  """Prepares target features (i.e., labels) from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  # Scale the target to be in units of thousands of dollars.
  output_targets["median_house_value"] = (
    california_housing_dataframe["median_house_value"] / 1000.0)
  return output_targets

In [4]:
# Choose the first 12000 (out of 17000) examples for training.
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_targets = preprocess_targets(california_housing_dataframe.head(12000))

# Choose the last 5000 (out of 17000) examples for validation.
validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())

Training examples summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0
mean,35.6,-119.6,28.6,2652.0,541.5,1428.8,502.8,3.9,2.0
std,2.1,2.0,12.6,2184.9,423.4,1103.4,386.0,1.9,1.1
min,32.5,-124.3,1.0,2.0,2.0,3.0,2.0,0.5,0.0
25%,33.9,-121.8,18.0,1465.8,296.0,789.8,281.0,2.6,1.5
50%,34.3,-118.5,29.0,2132.5,435.0,1168.0,410.0,3.6,1.9
75%,37.7,-118.0,37.0,3160.0,651.0,1727.0,607.0,4.8,2.3
max,42.0,-114.3,52.0,37937.0,5471.0,16122.0,5189.0,15.0,41.3


Validation examples summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0
mean,35.6,-119.5,28.5,2623.7,534.3,1431.5,497.4,3.9,2.0
std,2.1,2.0,12.6,2168.2,416.9,1248.3,380.9,1.9,1.3
min,32.5,-124.3,2.0,8.0,1.0,9.0,1.0,0.5,0.1
25%,33.9,-121.8,18.0,1448.8,297.8,790.8,283.0,2.5,1.5
50%,34.2,-118.5,29.0,2114.0,430.0,1163.5,405.0,3.5,1.9
75%,37.7,-118.0,37.0,3136.2,646.0,1714.0,602.0,4.8,2.3
max,41.8,-114.6,52.0,32627.0,6445.0,35682.0,6082.0,15.0,55.2


Training targets summary:


,median_house_value
count,12000.0
mean,206.6
std,114.9
min,15.0
25%,120.1
50%,179.6
75%,264.6
max,500.0


Validation targets summary:


,median_house_value
count,5000.0
mean,208.9
std,118.6
min,15.0
25%,118.5
50%,181.4
75%,265.7
max,500.0


In [5]:
def construct_feature_columns(input_features):
    """Construct the TesnorFlow Feature Columns
    
    Args:
        input_features: The names of the numerical input features to use.
    Returns:
        A set of feature columns
    """
    return set([tf.feature_column.numeric_column(my_feature)
               for my_feature in input_features])

In [6]:
def my_input_fn(features, targests, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value)  for key,value in dict(features).items()}
    
    # Construct a dataset, and configure batching/repeating
    ds = Dataset.from_tensor_Slices((features, targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        ds = ds.shuffle(10000)
        
    features, labels = ds.make_one_shot_iterator().get_next()
    
    return features, labels

## FTRL Optmization Algorithm

High dimensional linear models can use a vairant of gradient-based optimization called FTRL. **It has a benefit that it scales the learning rate differently for different coefficients**, which can be useful if some features rarely take non-zero values.

In [ ]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    feature_columns,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets
):
    """Trains a linear regression model.
    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.
  
    Args:
        learning_rate: A `float`, the learning rate.
        steps: A non-zero `int`, the total number of training steps. A training step
          consists of a forward and backward pass using a single batch.
        feature_columns: A `set` specifying the input feature columns to use.
        training_examples: A `DataFrame` containing one or more columns from
          `california_housing_dataframe` to use as input features for training.
        training_targets: A `DataFrame` containing exactly one column from
          `california_housing_dataframe` to use as target for training.
        validation_examples: A `DataFrame` containing one or more columns from
          `california_housing_dataframe` to use as input features for validation.
        validation_targets: A `DataFrame` containing exactly one column from
          `california_housing_dataframe` to use as target for validation.
      
    Returns:
        A `LinearRegressor` object trained on the training data.
    """
    periods = 10
    steps_per_period = steps / periods
    
    # Modified from exercise to work with new TF
    my_optimizer = partial(optimizers.FTRL, lr=learning_rate, clipvalue=0.5)
    linear_regressor = tf.estimator.LinearRegressor(
        feature_columns=construct_feature_columns(training_examples),
        optimizer=my_optimizer
    )
    
    training_input_fn = lambda: my_input_fn(training_examples, 
                                            training_targets["median_house_value"], 
                                             batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                    training_targets["median_house_value"], 
                                                    num_epochs=1, 
                                                    shuffle=False)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                      validation_targets["median_house_value"], 
                                                      num_epochs=1, 
                                                      shuffle=False)
    
    # Train the model
    print("Training model...")
    print("RMSE (on training data):")
    
    training_rmse = []
    
    